In [180]:
import numpy as np
import pandas as pd

In [181]:
IB_data = pd.read_csv("datas/MYM/IB合併_5min_OHLCV-20240701-20241030.csv")
data = IB_data[["datetime", "open", "high", "low", "close", "volume"]]
data.head()

,datetime,open,high,low,close,volume
0,2024-05-27 15:00:00,39553.0,39553.0,39553.0,39553.0,0
1,2024-05-27 15:05:00,39548.0,39548.0,39548.0,39548.0,2
2,2024-05-27 15:10:00,39548.0,39548.0,39548.0,39548.0,0
3,2024-05-27 15:15:00,39543.0,39543.0,39543.0,39543.0,1
4,2024-05-27 15:20:00,39543.0,39543.0,39543.0,39543.0,0


##### 從MYM的量價製作技術分析欄位

In [182]:
from datetime import datetime, time


def add_technical_indicators(df):
    """
    添加技術指標到DataFrame

    Parameters:
    df: 包含必要欄位的DataFrame (至少需要 datetime, open, high, low, close, volume)

    Returns:
    DataFrame: 包含所有技術指標的DataFrame
    """
    # 複製DataFrame以避免修改原始數據
    df = df.copy()

    # 確保datetime欄位為datetime格式
    df["datetime"] = pd.to_datetime(df["datetime"])

    # 移動平均線及其斜率
    for minutes in [15, 30, 60,120, 360, 1440,7200]:
        period = minutes//5
        df[f"mym_ma_{minutes}"] = df["close"].rolling(window=period).mean()
        df[f"mym_ma_slope_{minutes}"] = df[f"mym_ma_{minutes}"].diff()  # 價格MA的斜率

    # 成交量相關指標及其斜率
    if "volume" in df.columns:
        for minutes in [15, 30, 60, 120, 360, 720, 1440, 7200]:
            period = minutes//5
            df[f"mym_volume_ma_{minutes}"] = df["volume"].rolling(window=period).mean()
            df[f"mym_volume_ma_slope_{minutes}"] = df[
                f"mym_volume_ma_{minutes}"
            ].diff()  # 成交量MA的斜率

    # VWAP
    df["vwap_cumulative_volume"] = df["volume"].cumsum()
    df["vwap_cumulative_volume_price"] = (df["close"] * df["volume"]).cumsum()
    df["VWAP"] = df["vwap_cumulative_volume_price"] / df["vwap_cumulative_volume"]

    for minutes in [30, 60]:
        period = minutes // 5
        df[f"VWAP_{minutes}漲跌幅"] = (df["VWAP"] / df["VWAP"].shift(period) - 1) * 100

    # ATR指標
    df["tr1"] = df["high"] - df["low"]
    df["tr2"] = abs(df["high"] - df["close"].shift())
    df["tr3"] = abs(df["low"] - df["close"].shift())
    df["TR"] = df[["tr1", "tr2", "tr3"]].max(axis=1)
    period = 14
    for period in [14, 21, 28,42]:
        df[f"ATR_{period}"] = df["TR"].rolling(window=period).mean()

        # 肯特納通道（KC）
        df["KC_middle"] = df["close"].rolling(window=period).mean()
        df["KC_upper"] = df["KC_middle"] + 2 * df[f"ATR_{period}"]
        df["KC_lower"] = df["KC_middle"] - 2 * df[f"ATR_{period}"]

    # OBV指標
    df["OBV"] = (np.sign(df["close"].diff()) * df["volume"]).fillna(0).cumsum()

    # KD指標
    periods = [14,28]
    for period in periods:
        df[f"{period}KD_lowest_low"] = df["low"].rolling(window=period).min()
        df[f"{period}KD_highest_high"] = df["high"].rolling(window=period).max()
        df[f"{period}%K"] = (
            (df["close"] - df[f"{period}KD_lowest_low"])
            / (df[f"{period}KD_highest_high"] - df[f"{period}KD_lowest_low"])
        ) * 100
        df[f"{period}KD_%D"] = df[f"{period}%K"].rolling(window=3).mean()

    # RSI
    delta = df["close"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df["RSI"] = 100 - (100 / (1 + rs))

    # MACD
    exp1 = df["close"].ewm(span=12, adjust=False).mean()
    exp2 = df["close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = exp1 - exp2
    df["Signal_Line"] = df["MACD"].ewm(span=9, adjust=False).mean()
    df["MACD_Histogram"] = df["MACD"] - df["Signal_Line"]

    # 布林通道
    period = 20
    df["BB_middle"] = df["close"].rolling(window=period).mean()
    std = df["close"].rolling(window=period).std()
    df["BB_upper"] = df["BB_middle"] + (std * 2)
    df["BB_lower"] = df["BB_middle"] - (std * 2)

    # 計算不同時間區間的漲跌幅
    for minutes in [15, 30, 60, 120, 240]:
        periods = minutes // 5  # 假設每個row是5分鐘
        # 計算之前時間的close到現在的漲跌幅
        df[f"return_{minutes}m"] = (df["close"] .diff(periods)/ df["close"].shift(periods)) * 100
        # 計算之前時間的close到現在的震幅
        df[f"return_{minutes}m"] = (
            df["close"].rolling(window=periods).std() / df["close"].shift(periods)
        ) * 100

        # 計算區間內最高價到現在的跌幅
        df[f"high_return_{minutes}m"] = (
            df["close"] / df["high"].rolling(window=periods).max() - 1
        ) * 100

        # 計算區間內最低價到現在的漲幅
        df[f"low_return_{minutes}m"] = (
            df["close"] / df["low"].rolling(window=periods).min() - 1
        ) * 100

    # 計算交易分鐘數
    def get_minutes_from_open(dt):
        if pd.isna(dt):
            return np.nan

        # 如果時間在04:00之後，15:00之前，返回nan
        if time(4, 0) <= dt.time() < time(15, 0):
            return np.nan

        # 如果時間在15:00之後
        if dt.time() >= time(15, 0):
            return (dt - dt.replace(hour=15, minute=0, second=0)).total_seconds() / 60

        # 如果時間在00:00到04:00之間
        if dt.time() < time(4, 0):
            prev_day = dt.replace(hour=15, minute=0, second=0) - pd.Timedelta(days=1)
            return (dt - prev_day).total_seconds() / 60

    df["minutes_from_open"] = df["datetime"].apply(get_minutes_from_open)

    def minutes_from_2130(dt):
        if pd.isna(dt):
            return np.nan

        # 確定當天的 21:30
        target_time = dt.replace(hour=21, minute=30, second=0, microsecond=0)

        # 計算與 21:30 的時間差（分鐘）
        time_difference = (dt - target_time).total_seconds() / 60

        return time_difference

    df["距離2130的時間差"] = df["datetime"].apply(minutes_from_2130)
    # 移除不必要的中間計算欄位
    df.drop(["tr1", "tr2", "tr3", "TR", "KC_middle","BB_middle"], axis=1, inplace=True)

    return df

# 假設你的DataFrame叫做df
result_df = add_technical_indicators(data)
result_df.iloc[500:505]

,datetime,open,high,low,close,volume,mym_ma_15,mym_ma_slope_15,mym_ma_30,mym_ma_slope_30,...,high_return_60m,low_return_60m,return_120m,high_return_120m,low_return_120m,return_240m,high_return_240m,low_return_240m,minutes_from_open,距離2130的時間差
500,2024-05-30 20:30:00,38600.0,38649.0,38600.0,38611.0,10,38597.000000,3.666667,38594.000000,4.000000,...,-0.098321,0.088135,0.024099,-0.098321,0.088135,0.043458,-0.098321,0.088135,330.0,-60.0
501,2024-05-30 20:35:00,38630.0,38630.0,38628.0,38628.0,6,38606.333333,9.333333,38600.833333,6.833333,...,-0.054335,0.132203,0.028132,-0.054335,0.132203,0.043145,-0.054335,0.132203,335.0,-55.0
502,2024-05-30 20:40:00,38622.0,38635.0,38622.0,38635.0,3,38624.666667,18.333333,38609.000000,8.166667,...,-0.036223,0.150349,0.033457,-0.036223,0.150349,0.044124,-0.036223,0.150349,340.0,-50.0
503,2024-05-30 20:45:00,38625.0,38628.0,38621.0,38621.0,5,38628.000000,3.333333,38612.500000,3.500000,...,-0.072447,0.114058,0.034072,-0.072447,0.114058,0.044168,-0.072447,0.114058,345.0,-45.0
504,2024-05-30 20:50:00,38608.0,38608.0,38603.0,38603.0,3,38619.666667,-8.333333,38613.000000,0.500000,...,-0.119020,0.067398,0.034096,-0.119020,0.067398,0.043900,-0.119020,0.067398,350.0,-40.0


標記答案(未來N分鐘漲跌幅)

In [183]:
import pandas as pd
import numpy as np


def calculate_future_changes(df):
    """
    計算未來不同時間區間的漲跌幅並進行分類

    分類規則:
    - 漲跌幅 > 0.2% -> 2
    - 0.1% < 漲跌幅 <= 0.2% -> 1
    - 漲跌幅 <= 0.1% -> 0
    """
    # 確保 datetime 欄位是索引且為 datetime 格式
    if "datetime" in df.columns:
        df = df.set_index("datetime")
    df.index = pd.to_datetime(df.index)

    # 定義要計算的時間區間（分鐘）
    time_periods = [5, 10, 15, 30, 60,90,120,180,240]

    for minutes in time_periods:
        # 計算未來價格
        future_price = df["close"].shift(-minutes)

        # 計算漲跌幅 (以百分比表示)
        pct_change = ((future_price - df["close"]) / df["close"]) * 10000

        # 根據條件分類
        # conditions = [
        #     (pct_change > 0.1),
        #     (pct_change > 0.05) & (pct_change <= 0.1),
        #     (pct_change <= 0.05) & (pct_change > -0.05),
        #     (pct_change < -0.05) & (pct_change >= -0.1),
        #     (pct_change < -0.1),
        # ]
        # choices = [2, 1, 0,-1,-2]
        # 0.2太過少見 改用0.1
        # threshold=0.06
        # conditions = [
        #     (pct_change > threshold),
        #     (pct_change <= threshold) & (pct_change > -threshold),
        #     (pct_change < -threshold),
        # ]
        # choices = [1, 0, -1]

        # 0.2太過少見 改用0.1
        # threshold = 0
        # conditions = [
        #     (pct_change > threshold),
        #     (pct_change <= -threshold),
        # ]
        # choices = [1,  -1]

        # 建立新欄位名稱
        column_name = f"future_{minutes}min_change"

        # # 使用 numpy.select 進行分類
        # df[column_name] = np.select(conditions, choices, default=np.nan)
        # df = df[df[column_name].notna()]
        df[column_name] =  pct_change

    return df


# 使用範例
# 假設你的數據已經讀入為 df

result_df_tags = calculate_future_changes(result_df)
result_df_tags.iloc[500:505]

,open,high,low,close,volume,mym_ma_15,mym_ma_slope_15,mym_ma_30,mym_ma_slope_30,mym_ma_60,...,距離2130的時間差,future_5min_change,future_10min_change,future_15min_change,future_30min_change,future_60min_change,future_90min_change,future_120min_change,future_180min_change,future_240min_change
datetime,,,,,,,,,,,,,,,,,,,,,
2024-05-30 20:30:00,38600.0,38649.0,38600.0,38611.0,10,38597.000000,3.666667,38594.000000,4.000000,38593.916667,...,-60.0,-2.071948,-3.366916,-9.841755,-6.733832,14.762632,2.071948,-0.776981,8.028800,74.849136
2024-05-30 20:35:00,38630.0,38630.0,38628.0,38628.0,6,38606.333333,9.333333,38600.833333,6.833333,38596.416667,...,-55.0,-0.258880,-7.248628,-4.142073,-17.086052,15.532774,-0.258880,-5.177591,5.436471,60.060060
2024-05-30 20:40:00,38622.0,38635.0,38622.0,38635.0,3,38624.666667,18.333333,38609.000000,8.166667,38599.500000,...,-50.0,-9.059143,11.647470,-13.976964,-8.282645,15.788793,-2.070661,-6.988482,2.847159,58.237350
2024-05-30 20:45:00,38625.0,38628.0,38621.0,38621.0,5,38628.000000,3.333333,38612.500000,3.500000,38601.583333,...,-45.0,-8.026721,-0.517853,-3.883897,1.812485,17.607001,7.508868,-5.955309,4.401750,85.186815
2024-05-30 20:50:00,38608.0,38608.0,38603.0,38603.0,3,38619.666667,-8.333333,38613.000000,0.500000,38602.083333,...,-40.0,-1.295236,5.699039,2.590472,0.518094,17.356164,8.548558,1.813331,6.994275,105.173173


In [184]:
# 將[ future_5min_change	future_10min_change	future_15min_change	future_30min_change	future_60min_change] 這幾個column挪到最前面
# 假設 data 是你的 DataFrame
# 設定要移動到最前面的 column
columns_to_move = [
    "future_5min_change",
    "future_10min_change",
    "future_15min_change",
    "future_30min_change",
    "future_60min_change",
    "future_90min_change",
    "future_120min_change",
    "future_180min_change",
    "future_240min_change",
]

# 獲取 DataFrame 中其餘的 column
remaining_columns = [
    col for col in result_df_tags.columns if col not in columns_to_move
]

# 重新排列 column，將指定的 column 移到最前面
result_df_tags_moved = result_df_tags[columns_to_move + remaining_columns]

# 確認新的 column 排列順序
result_df_tags_moved.columns

Index(['future_5min_change', 'future_10min_change', 'future_15min_change',
       'future_30min_change', 'future_60min_change', 'future_90min_change',
       'future_120min_change', 'future_180min_change', 'future_240min_change',
       'open', 'high', 'low', 'close', 'volume', 'mym_ma_15',
       'mym_ma_slope_15', 'mym_ma_30', 'mym_ma_slope_30', 'mym_ma_60',
       'mym_ma_slope_60', 'mym_ma_120', 'mym_ma_slope_120', 'mym_ma_360',
       'mym_ma_slope_360', 'mym_ma_1440', 'mym_ma_slope_1440', 'mym_ma_7200',
       'mym_ma_slope_7200', 'mym_volume_ma_15', 'mym_volume_ma_slope_15',
       'mym_volume_ma_30', 'mym_volume_ma_slope_30', 'mym_volume_ma_60',
       'mym_volume_ma_slope_60', 'mym_volume_ma_120',
       'mym_volume_ma_slope_120', 'mym_volume_ma_360',
       'mym_volume_ma_slope_360', 'mym_volume_ma_720',
       'mym_volume_ma_slope_720', 'mym_volume_ma_1440',
       'mym_volume_ma_slope_1440', 'mym_volume_ma_7200',
       'mym_volume_ma_slope_7200', 'vwap_cumulative_volume',


In [149]:
result_df_tags_moved.to_csv("datas/5min_MYM_只用IB量價資料_class.csv")

In [185]:
result_df_tags_moved.to_csv("datas/5min_MYM_只用IB量價資料_regression.csv")

然後繼續加入YM的資料

In [186]:
# df_origin = pd.read_csv("datas/5min_MYM_只用IB量價資料_class.csv")
df_origin = pd.read_csv("datas/5min_MYM_只用IB量價資料_regression.csv")

In [187]:
df_YM = pd.read_csv("datas/YM/YM_0701-1030_5min含筆數與張數.csv")
df_origin = df_origin.drop_duplicates(subset=["datetime"])
df_YM = df_YM.drop_duplicates(subset=["datetime"])
df_addYM = df_origin.merge(df_YM, on="datetime", how="left")
# df_addYM

再加入ES的資料

In [188]:
df_ES = pd.read_csv("datas/ES/ES_0701-1030_5min含筆數與張數.csv")
df_addES = df_addYM.merge(df_ES, on="datetime", how="left")
# df_addES

再加入VIX的資料

In [189]:
df_VIX近 = pd.read_csv("datas/VIX/IB_VIX近_final.csv")
df_addVIX = df_addES.merge(df_VIX近, on="datetime", how="left")
df_addVIX = df_addVIX.drop(["Unnamed: 0"], axis=1)
# df_addVIX

In [153]:
df_addVIX.to_csv("datas/5min_MYM_量價+YM+ES+VIX資料_class.csv")

加入ZN(10年公債)

In [190]:
df_ZN = pd.read_csv("datas/ZN/ZN_0701-1030_5min含筆數與張數.csv")
df_addZN = df_addVIX.merge(df_ZN, on="datetime", how="left")

加入RTY(羅素)

In [191]:
df_RTY = pd.read_csv("datas/RTY/RTY_0701-1030_5min含筆數與張數.csv")
df_addRTY = df_addZN.merge(df_RTY, on="datetime", how="left")

加入NQ

In [192]:
df_NQ = pd.read_csv("datas/NQ/NQ_0701-1030_5min含筆數與張數.csv")
df_addNQ = df_addRTY.merge(df_NQ, on="datetime", how="left")

加入DX(美元指數)

In [193]:
df_DX = pd.read_csv("datas/DX/DX_0701-1030_5min含筆數與張數.csv")
df_addDX = df_addNQ.merge(df_DX, on="datetime", how="left")

In [194]:
df_addDX.columns

Index(['datetime', 'future_5min_change', 'future_10min_change',
       'future_15min_change', 'future_30min_change', 'future_60min_change',
       'future_90min_change', 'future_120min_change', 'future_180min_change',
       'future_240min_change',
       ...
       'NQ close', 'NQ volume', 'NQ count', 'NQ 30分鐘漲跌幅', 'NQ 60分鐘漲跌幅',
       'DX close', 'DX volume', 'DX count', 'DX 30分鐘漲跌幅', 'DX 60分鐘漲跌幅'],
      dtype='object', length=140)

In [140]:
df_addDX.to_csv("datas/5min_MYM_量價+YM+ES+VIX+ZN+RTY+NQ+DX資料_class.csv")
# df_addVIX.to_csv("datas/5min_MYM_量價+YM+ES+VIX資料_class.csv")

In [195]:
df_addDX.to_csv("datas/5min_MYM_量價+YM+ES+VIX+ZN+RTY+NQ+DX資料_regression.csv")